In [1]:
!pip install plotly

!pip install optuna

!pip install lightgbm

!pip install kaleido

!pip install pyarrow
# 科学计算模块
import matplotlib
import numpy as np
import pandas as pd

# 绘图模块
import matplotlib as mpl
import matplotlib.pyplot as plt
import plotly
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score
from optuna.importance import MeanDecreaseImpurityImportanceEvaluator
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler
# Scikit-Learn相关模块
# 评估器类
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
import numpy as np
import optuna
import lightgbm as lgb
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split
# 实用函数
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import lightgbm as lgb
from lightgbm import LGBMClassifier


import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from tqdm import tqdm

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [2]:
def optimized_preprocessing(df, int):
    cat_features = ["B_30", "B_38", "D_114", "D_116", "D_117", "D_120", "D_126", "D_63", "D_64", "D_66", "D_68"]
    ignore_features = ["B_30", "B_38", "D_114", "D_116", "D_117", "D_120", "D_126", "D_63", "D_64", "D_66", "D_68",
                       'S_2', 'customer_ID','target']
    numeric_features = [col for col in df.columns if col not in ignore_features]
    target_features = ['target']
    
    
    #print("This is in [2], Columns before processing:"+",".join(df.columns))
    
    
    # 使用更高效的方式填充缺失值
    df[numeric_features] = df[numeric_features].fillna(df[numeric_features].median())
    df[cat_features] = df[cat_features].fillna(df[cat_features].mode().iloc[0])

    # 类别特征的独热编码
    encoder = OneHotEncoder()
    encoded_data = encoder.fit_transform(df[cat_features]).toarray()
    new_feature_names = encoder.get_feature_names_out(cat_features)
    encoded_df = pd.DataFrame(encoded_data, columns=new_feature_names)
    final_df = pd.concat([df.drop(cat_features, axis=1), encoded_df], axis=1)

    # 数值特征处理
    final_df[numeric_features] = np.floor(df[numeric_features] * 100)

    # 时间特征处理
    final_df["S_2"] = pd.to_datetime(df["S_2"])

    # 聚合特征
    if int == 0:
        agg_funcs = {
        "S_2": ['count'],
        **{name: ['last', 'nunique'] for name in new_feature_names},
        **{name: ['mean', 'std', 'min', 'max'] for name in numeric_features},
        label_name: ['last']  # 添加对标签的聚合操作
        }
    else:
        agg_funcs ={
        "S_2": ['count'],
        **{name: ['last', 'nunique'] for name in new_feature_names},
        **{name: ['mean', 'std', 'min', 'max'] for name in numeric_features},
        }
        

    final_agg_df = final_df.groupby("customer_ID", sort=False).agg(agg_funcs).reset_index(drop=False)
    
    new_feature_names=['_'.join(x) for x in final_agg_df.columns]
    final_agg_df.columns = new_feature_names
    # 如果 'target_last' 存在于列名中，将其重命名为 'target'
    
    
    if 'target_last' in final_agg_df.columns:
        final_agg_df = final_agg_df.rename(columns={'target_last': 'target'})
    
    if 'customer_ID_'in final_agg_df.columns:
        final_agg_df = final_agg_df.rename(columns={'customer_ID_': 'customer_ID'})
    
    customer_ids = final_agg_df['customer_ID']
    final_agg_df.reset_index(drop=False,inplace = True)
    final_agg_df=final_agg_df.drop(columns=['customer_ID'])
    
    #print("This is in [2], final_agg_df.columns: "+",".join(final_agg_df.columns))
    
    return final_agg_df,new_feature_names,customer_ids


In [3]:
def create_objective(train_data,new_feature_names):    
    def objective(trial):
        
        kf = KFold(n_splits=5, shuffle=True, random_state=42)
        accuracies = []
        for train_index, test_index in kf.split(train_data):
            
            #print("train_data.columns in kf loop: "+','.join(train_data.columns))
            X_train_fold = train_data.iloc[train_index].drop(columns=['target'])
            y_train_fold = train_data.iloc[train_index]['target']
            X_valid_fold = train_data.iloc[test_index].drop(columns=['target'])
            y_valid_fold = train_data.iloc[test_index]['target']

            dtrain = lgb.Dataset(X_train_fold, label=y_train_fold)

            #print("X_train_fold columns: "+ ','.join(X_train_fold.columns))
            param = {

                "objective": "binary",
                "metric": "binary_logloss",
                "verbosity": -1,
                "boosting_type": "gbdt",
                "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
                "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
                "num_leaves": trial.suggest_int("num_leaves", 2, 256),
                "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
                "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
                "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
                "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
                "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
                "max_depth": trial.suggest_int("max_depth", 3, 10),
                "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 200),
                "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 1.0),
                #"categorical_feature": ["name:" + col for col in new_feature_names if col != "target"]
            }

            # 训练模型
            gbm = lgb.train(param, dtrain)

            # 获取验证集上的预测概率
            probs = gbm.predict(X_valid_fold)
            pred_labels = (probs >= 0.5).astype(int)

            # 计算精度
            accuracy = accuracy_score(y_valid_fold, pred_labels)
            accuracies.append(accuracy)

            # 返回交叉验证的平均精度
        return np.mean(accuracies)
    return objective

In [4]:
train_data_original = pd.read_feather('./input/train.feather')
labels = pd.read_csv('/openbayes/input/input0/train_labels.csv')
test_data = pd.read_feather('./input/test.feather')
# 保留原有顺序，仅删除连续重复的 customer_ID
#customer_ids = test_data['customer_ID'].drop_duplicates(keep='first') 


In [5]:
print(labels.columns)

train_data = pd.merge(train_data_original, labels, on='customer_ID', how='left')  # 合并标签列到训练数据集中
print(train_data.columns)

label_name = 'target'

Index(['customer_ID', 'target'], dtype='object')
Index(['customer_ID', 'S_2', 'P_2', 'D_39', 'B_1', 'B_2', 'R_1', 'S_3', 'D_41',
       'B_3',
       ...
       'D_137', 'D_138', 'D_139', 'D_140', 'D_141', 'D_142', 'D_143', 'D_144',
       'D_145', 'target'],
      dtype='object', length=191)


In [6]:
train_data,new_feature_names,_ = optimized_preprocessing(train_data, 0)
# 移除 'customer_ID_'
#print("***********")
#print(new_feature_names)
if 'customer_ID_' in new_feature_names:
    new_feature_names.remove('customer_ID_')
if 'S_2_count' in new_feature_names:
    new_feature_names.remove('S_2_count')


# 将 'target_last' 替换为 'target'
if 'target_last' in new_feature_names:
    index = new_feature_names.index('target_last')
    new_feature_names[index] = 'target'
    
#print(new_feature_names)

#print("***********")
test_data,useless,customer_ids = optimized_preprocessing(test_data, 1)

#print("This is in [6]，train_data after preprocessing is: " + ', '.join(train_data.columns))

In [7]:
evaluator = MeanDecreaseImpurityImportanceEvaluator()
study = optuna.create_study(
    direction="maximize",
    sampler=TPESampler(),
    pruner=MedianPruner()
)
print("study begin")
print(train_data.columns)
#train_data=train_data.drop(columns=['customer_ID'])
objective = create_objective(train_data,new_feature_names)
study.optimize(objective, n_trials=100)
# optuna.visualization.plot_contour(study)#可视化
if (optuna.visualization.is_available() == True):
    print("available")

fig1 = optuna.visualization.plot_optimization_history(study)  # 画出一个 study 中所有 trial 的优化历史记录。
fig1.write_image("optimization_history.png")

# fig2 =optuna.visualization.plot_parallel_coordinate(study)#绘制一个 study 中高维度参数的关系图。
# fig2.write_image("parallel_coordinate.png")

fig3 = optuna.visualization.plot_param_importances(study, evaluator=evaluator)  # 画出超参数的重要性
fig3.write_image("param_importances.png")

# fig4=optuna.visualization.plot_contour(study) #将参数关系画成等高线图。
# fig4.write_image("contour.png")

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial: {}")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))



[I 2023-12-24 08:37:13,944] A new study created in memory with name: no-name-dc4e7bf0-085d-4f7a-bb68-98cbc66133a8


study begin
Index(['index', 'S_2_count', 'B_30_0.0_last', 'B_30_0.0_nunique',
       'B_30_100.0_last', 'B_30_100.0_nunique', 'B_30_200.0_last',
       'B_30_200.0_nunique', 'B_38_100.0_last', 'B_38_100.0_nunique',
       ...
       'D_143_max', 'D_144_mean', 'D_144_std', 'D_144_min', 'D_144_max',
       'D_145_mean', 'D_145_std', 'D_145_min', 'D_145_max', 'target'],
      dtype='object', length=803)


[I 2023-12-24 08:39:54,976] Trial 0 finished with value: 0.8940474525332694 and parameters: {'lambda_l1': 1.9284109036662538e-08, 'lambda_l2': 1.11009704312992e-08, 'num_leaves': 143, 'feature_fraction': 0.4845486440170327, 'bagging_fraction': 0.4862015813301669, 'bagging_freq': 6, 'min_child_samples': 14, 'learning_rate': 0.24482838336293292, 'max_depth': 7, 'min_data_in_leaf': 189, 'colsample_bytree': 0.8068620812441352}. Best is trial 0 with value: 0.8940474525332694.
[I 2023-12-24 08:41:51,108] Trial 1 finished with value: 0.8956664960981312 and parameters: {'lambda_l1': 0.00021073209456348493, 'lambda_l2': 0.012069795699759293, 'num_leaves': 226, 'feature_fraction': 0.5187142575449699, 'bagging_fraction': 0.6672251993530568, 'bagging_freq': 7, 'min_child_samples': 45, 'learning_rate': 0.15702204826078422, 'max_depth': 5, 'min_data_in_leaf': 121, 'colsample_bytree': 0.41897423095007075}. Best is trial 1 with value: 0.8956664960981312.
[I 2023-12-24 08:44:34,515] Trial 2 finished wi

available
Number of finished trials: 100
Best trial: {}
  Value: 0.8971526147377377
  Params: 
    lambda_l1: 0.0001220450790990738
    lambda_l2: 0.00029227171778729676
    num_leaves: 91
    feature_fraction: 0.9824939469789891
    bagging_fraction: 0.982871483805669
    bagging_freq: 3
    min_child_samples: 55
    learning_rate: 0.11838275960083297
    max_depth: 10
    min_data_in_leaf: 94
    colsample_bytree: 0.9513279658945938


In [8]:
print(train_data.columns)

Index(['index', 'S_2_count', 'B_30_0.0_last', 'B_30_0.0_nunique',
       'B_30_100.0_last', 'B_30_100.0_nunique', 'B_30_200.0_last',
       'B_30_200.0_nunique', 'B_38_100.0_last', 'B_38_100.0_nunique',
       ...
       'D_143_max', 'D_144_mean', 'D_144_std', 'D_144_min', 'D_144_max',
       'D_145_mean', 'D_145_std', 'D_145_min', 'D_145_max', 'target'],
      dtype='object', length=803)


In [9]:
# 找出训练数据和测试数据中特征的不一致
train_features = set(train_data.columns)
test_features = set(test_data.columns)

# 在训练数据中存在但在测试数据中不存在的特征
missing_in_test = train_features - test_features

# 在测试数据中存在但在训练数据中不存在的特征
missing_in_train = test_features - train_features

# 输出缺失的特征
print("Missing in Test: ", missing_in_test)
print("Missing in Train: ", missing_in_train)

# 处理缺失的特征
# 您可以根据具体情况选择添加缺失的特征或删除多余的特征
# 为测试集中缺失的特征添加默认值
for feature in missing_in_test:
    if feature != 'target':  # 排除 'target'，因为它不应该在测试集中
        # 用适当的默认值填充，这里用0作为例子
        test_data[feature] = 0


Missing in Test:  {'D_66_0.0_nunique', 'D_64_1_nunique', 'D_66_0.0_last', 'D_64_1_last', 'D_68_0.0_nunique', 'D_68_0.0_last', 'target'}
Missing in Train:  set()


In [10]:
# 使用最佳参数重新训练模型
best_params = trial.params
best_model = lgb.train(best_params, lgb.Dataset(train_data.drop(columns=["target"]), label=train_data["target"]))

# 对测试数据进行概率预测
test_probs = best_model.predict(test_data)


[LightGBM] [Warning] feature_fraction is set=0.9824939469789891, colsample_bytree=0.9513279658945938 will be ignored. Current value: feature_fraction=0.9824939469789891
[LightGBM] [Warning] min_data_in_leaf is set=94, min_child_samples=55 will be ignored. Current value: min_data_in_leaf=94
[LightGBM] [Warning] feature_fraction is set=0.9824939469789891, colsample_bytree=0.9513279658945938 will be ignored. Current value: feature_fraction=0.9824939469789891
[LightGBM] [Warning] min_data_in_leaf is set=94, min_child_samples=55 will be ignored. Current value: min_data_in_leaf=94
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.774178 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 100360
[LightGBM] [Info] Number of data points in the train set: 458913, number of used features: 783
[LightGBM] [Info] Start training from score 0.258934


In [11]:
print(test_probs)
print(test_probs.shape)

[0.60837108 0.53860544 0.52500035 ... 0.60213668 0.86065507 0.46585182]
(924621,)


In [12]:

print(customer_ids)

0         00000469ba478561f23a92a868bd366de6f6527a684c9a...
1         00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...
2         0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...
3         00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...
4         00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...
                                ...                        
924616    ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...
924617    ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...
924618    ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475...
924619    ffffddef1fc3643ea179c93245b68dca0f36941cd83977...
924620    fffffa7cf7e453e1acc6a1426475d5cb9400859f82ff61...
Name: customer_ID, Length: 924621, dtype: object


In [13]:


# 创建一个包含 customer_ID 和预测概率的 DataFrame
results_df = pd.DataFrame({
    'customer_ID': customer_ids,
    'prediction': test_probs
})

# 将 DataFrame 保存为 CSV 文件
result_csv=results_df.to_csv('predictions.csv', index=False)

### 

In [14]:
print(best_params)

{'lambda_l1': 0.0001220450790990738, 'lambda_l2': 0.00029227171778729676, 'num_leaves': 91, 'feature_fraction': 0.9824939469789891, 'bagging_fraction': 0.982871483805669, 'bagging_freq': 3, 'min_child_samples': 55, 'learning_rate': 0.11838275960083297, 'max_depth': 10, 'min_data_in_leaf': 94, 'colsample_bytree': 0.9513279658945938}


In [16]:
print(best_model.params)

{'lambda_l1': 0.0001220450790990738, 'lambda_l2': 0.00029227171778729676, 'num_leaves': 91, 'feature_fraction': 0.9824939469789891, 'bagging_fraction': 0.982871483805669, 'bagging_freq': 3, 'min_child_samples': 55, 'learning_rate': 0.11838275960083297, 'max_depth': 10, 'min_data_in_leaf': 94, 'colsample_bytree': 0.9513279658945938, 'objective': None, 'num_iterations': 100}
